# Baseline Multi-Perceptron Model

## Summary

## Table of Contents

- [Notebook Setup](#Notebook-Setup)
- [Read in Parquet](#Read-in-Parquet)
- [MLP Baseline Model](#Encode-Features)
- [Results](#Results)
- [Save as Pickle](#Save-as_Pickle)

## Notebook Setup

Significant functions from [assignment_3_tools.py](./assignment_3_tools.py)

In [1]:
import os
import pickle #for saveing and loading trained models
import numpy as np # for vector / matrix operations
import pandas as pd # for data manipulation
import seaborn as sns # For plots
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, make_scorer, recall_score
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier as mlp
from assignment_3_tools import parquet_to_dict

data_pth = "../../Data/GoogleDrive/Encoded_Data/"
save_pth = "../../Data/GoogleDrive/Baseline/"

## Read in Parquet

In [2]:
# Lazy read encoded data.
pq_jar = parquet_to_dict(data_pth)

# Unique Datasets.
def encode_dataset(lazy_dict):
    all_names = list()
    for i, key in enumerate(lazy_dict):
        if key[-6:] == "_train":
            all_names.append(key[:-8])
        elif key[-5:] == "_test":
            all_names.append(key[:-7])
        else:
            pass
    unq_names = set(all_names)
    return unq_names

unq_names = encode_dataset(pq_jar)

# Return Corresponding Test Set.
def corr_testset(unq_name):
    threshold = unq_name[-2:]
    if threshold.isnumeric():
        X_test_name = f"df_heart_drop_{threshold}_imp_X_test"
        y_test_name = f"df_heart_drop_{threshold}_imp_y_test"
    else:
        X_test_name = f"{unq_name}_X_test"
        y_test_name = f"{unq_name}_y_test"
    return X_test_name, y_test_name

## MLP Baseline Model

In [8]:
def mlp_baseline(lazy_dict):
    baseline_results = dict()
    # Initialize MLP parameters
    params = {'activation':'tanh', 'solver':'adam', 'valid_frac':0.2,
      'alpha':0.001, 'learn_rate_init':0.0001,
      'max_iter':1000, 'n_iter_no_change':200,
      'rand_state': 3}
    
    # MLP for each Dataframe
    for name in unq_names:
        # pl.LazyFrame Names
        X_train_name = f"{name}_X_train"
        y_train_name = f"{name}_y_train"
        (X_test_name, y_test_name) = corr_testset(name)
        
        # Collect pl.LazyFrame and convert to pd.DataFrame
        X_train = lazy_dict[X_train_name].collect().to_pandas()
        y_train = lazy_dict[y_train_name].collect().to_numpy().ravel()
        X_test = lazy_dict[X_test_name].collect().to_pandas()
        y_test = lazy_dict[y_test_name].collect().to_numpy().ravel()
        
        # Remove index column
        if "__index_level_0__" in X_train.columns:
            X_train = X_train.drop(columns=['__index_level_0__'])
        if "__index_level_0__" in X_test.columns:
            X_test = X_test.drop(columns=['__index_level_0__'])
            
        # Standardize X_train and X_test by the standardization scalar of X_train
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        print(f"Training MLP on {name}...")
        
        # Initialize mlp model
        mlp_model = mlp(
            hidden_layer_sizes=([8,4,8]),
            validation_fraction = params['valid_frac'],
            activation = params['activation'],
            solver = params['solver'],
            alpha = params['alpha'],
            learning_rate = "adaptive",
            learning_rate_init = params['learn_rate_init'],
            batch_size = "auto",
            max_iter = params['max_iter'],
            early_stopping = True,
            n_iter_no_change = params['n_iter_no_change'],
            verbose= False,
            random_state = params['rand_state'])
        
        # Train the model
        mlp_model.fit(X_train_scaled, y_train)
        
        # Save the Trained model
        with open(f"{save_pth}{name}.pkl", 'wb') as file:
            pickle.dump(mlp_model, file)
            
        # Cross-Validation and Evaluation
        scoring = {'recall': make_scorer(recall_score)}
        scores = cross_validate(mlp_model, X_train_scaled, y_train, cv=5, scoring=scoring, return_train_score=True)

        # Store results in the dictionary
        baseline_results['dataset_name'].append(name)
        baseline_results['train_recall'].append(scores['train_recall'].mean())
        baseline_results['test_recall'].append(scores['test_recall'].mean())
        baseline_results['fit_time'].append(scores['fit_time'].mean())
        print(f"Completed Training MLP on {name}!")
    df_baseline_results = pd.DataFrame(baseline_results)
    df_baseline_results.to_parquet(f"{save_pth}baseline_results.pkl")
    return df_baseline_results

mlp_baseline(pq_jar)

Training MLP on Over_Sample_1:3_threshold_40...


/opt/anaconda3/envs/203C/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

## Save as Pickle